# Dataset creation

## Installation

In [101]:
!pip install -q llama-cpp-python textdescriptives argilla==1.18 transformers datasets langdetect langchain


[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [102]:
!python -m spacy download en_core_web_md

Looking in indexes: https://pypi.org/simple, https://dmrepository.datamaran.com:8443/repository/dmPYTHON/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 41.9 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: en-core-web-md
    Found existing installation: en-core-web-md 3.7.0
    Uninstalling en-core-web-md-3.7.0:
      Successfully uninstalled en-core-web-md-3.7.0

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [103]:
!CMAKE_ARGS="-DLLAMA_METAL=on" FORCE_CMAKE=1 pip install --upgrade --force-reinstall llama-cpp-python --no-cache-dir

Looking in indexes: https://pypi.org/simple, https://dmrepository.datamaran.com:8443/repository/dmPYTHON/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 18.9 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 870.3 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 50.6 MB/s eta 0:00:00a 0:00:01
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.18-cp39-cp39-macosx_13_0_x86_64.whl size=1886255 sha256=635b308f295db89a312328747563c1229ae652af1940ec3a3cbd798cfd6e9562
  Stored in directory: /private/var/folders/9t/msy700h16jz3q35qvg4z1ln40000gn/T/pip-ephem-wheel-cache-y4tj1_uj/wheels/f5/c3/64/dceaadaf16197204d2344811b0c2b684c420f545bc0c27fad9
Successfully built llama-cpp-python
  Attempting uninstall: typing-extensio

In [ ]:
# from ctransformers import AutoModelForCausalLM

# # Set gpu_layers to the number of layers to offload to GPU. Set to 0 if no GPU acceleration is available on your system.
# llm = AutoModelForCausalLM.from_pretrained("TheBloke/Mistral-7B-Instruct-v0.1-GGUF", model_file="mistral-7b-instruct-v0.1.Q4_K_M.gguf", model_type="mistral")

# print(llm("AI is going to"))

## Imports

In [104]:
import textdescriptives as td
from datasets import load_dataset
import re
import spacy
from langdetect import detect
import argilla as rg
import numpy as np
import concurrent.futures
import requests
import json

/Users/davidberenstein/Documents/programming/argilla/awesome-argilla-datasets/.env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Connect to Argilla

In [114]:
import os
import argilla as rg
rg.init(api_url="https://argilla-quickstart-pr-4215-ki24f765kq-no.a.run.app", api_key="ResJl6fGZw8c4oJhagsJixG2u5xUVLwZ")
rg.set_workspace("admin")

/Users/davidberenstein/Documents/programming/argilla/awesome-argilla-datasets/.env/lib/python3.9/site-packages/argilla/client/client.py:165: UserWarning: No workspace configuration was detected. To work with Argilla datasets, specify a valid workspace name on `rg.init` or set it up through the `rg.set_workspace` function.
  warnings.warn(
/Users/davidberenstein/Documents/programming/argilla/awesome-argilla-datasets/.env/lib/python3.9/site-packages/argilla/client/client.py:182: UserWarning: You're connecting to Argilla Server 1.20.0-dev using a different client version (1.18.0).
This may lead to potential compatibility issues during your experience.
To ensure a seamless and optimized connection, we highly recommend aligning your client version with the server version.
  warnings.warn(


## Pre-processing

End-to-end workflow to create a dataset in Argilla with text measurements as metadata.
This aids in quickly identifying and improving potential dataset issues.

### Dataset creation

At first, we need to create a dataset in Argilla. This can either be done by loading a previous created dataset or by creating a new one. In order to avoid duplication, we will check if the dataset already exists. Additionally, we will load the markdown file that contains the dataset guidelines.


In [112]:
with open("GUIDELINES.md") as f:
    guidelines = f.read()
guidelines

"# Guidelines§\nThe dataset aims to classify three things:\n\n1. Quality\n2. Intent\n3. Toxicity\n\n## Quality\n\nFor the quality, we have decided to define a rating question on a scale from 1 to 7. This question is used to assess the quality of the prompt, based on quality, grammar and clarity of context.\n\n## Intent\n\nFor the intent, we have decided to define a multi-label classification question which will be used to determine the prompt types as defined in the [Llama 2 paper](https://arxiv.org/abs/2307.09288). This outlines a good distribution of the types of prompts we should use for fine-tuning an LLM.\n\n- Generation: A statement that indicates the need for open generation.\n- Rewrite: A statement that is rewritten by the model.\n- Extract: A question that is answered by extracted from the context in the prompt.\n- Closed QA: A question that is answered by the model based on the context.\n- Open QA: A question that is answered by the model without any context.\n- Classificatio

In [115]:
try:
    ds_local = rg.FeedbackDataset.for_supervised_fine_tuning(context=True, use_markdown=True, guidelines=guidelines)
    ds_local.questions.extend([
        rg.RatingQuestion(
            name="prompt-quality", 
            title="Prompt Quality",
            values=list(range(1, 8)), 
            description="How would you rate the quality of the prompt?",
        ),
        rg.LabelQuestion(
            name="prompt-intent", 
            title="Prompt Intent",
            labels=["generation", "rewrite", "extract", "closed-qa", "open-qa", "classification", "summarization", "brainstorming", "chat", "code", "other"], 
            description="What is the intent of the prompt?"
        ),
        rg.MultiLabelQuestion(
            name="prompt-toxicity", 
            title="Prompt Toxicity",
            labels=["illegal", "harmfull", "unqualified advice"], 
            description="What are the toxicities in the prompt (if any)?",
            required=False
        )
    ])
    ds_remote = ds_local.push_to_argilla("sharegpt")
except Exception as e:
    ds_remote = rg.FeedbackDataset.from_argilla("sharegpt")
ds_remote

RemoteFeedbackDataset(
   id=fca7b82d-cd72-4648-afef-6728e43c8597
   name=sharegpt
   workspace=Workspace(id=21f132b5-7efd-40f1-96a6-7c25d2672710, name=admin, inserted_at=2023-11-14 07:55:35.549938, updated_at=2023-11-14 07:55:35.549938)
   url=https://argilla-quickstart-pr-4215-ki24f765kq-no.a.run.app/dataset/fca7b82d-cd72-4648-afef-6728e43c8597/annotation-mode
   fields=[RemoteTextField(id=UUID('da59a668-8a93-4aaf-a1b3-050927b60f0e'), client=None, name='prompt', title='Prompt', required=True, type='text', use_markdown=True), RemoteTextField(id=UUID('47adb7af-4565-4676-b573-3521ce3b60aa'), client=None, name='context', title='Context', required=False, type='text', use_markdown=True)]
   questions=[RemoteTextQuestion(id=UUID('17ba93e0-f190-4745-a2f1-506010ac7bfb'), client=None, name='response', title='Response', description=None, required=True, type='text', use_markdown=True), RemoteRatingQuestion(id=UUID('f3850d16-ef13-433e-bf4b-546ab5f2faf8'), client=None, name='prompt-quality', title

#### Configure the metadata-properties

Next we will be using `text-descriptives` to configure the metadata-properties. This will be used to add and updat relevant metadata-properties to the dataset. Because `text-descriptives` doesn't provide any programmatic interface with the metrics-groups and their sub-metrics, we will run the computation on the an example text and use the results to configure the metadata-properties.

In [116]:
metric_group = ["descriptive_stats"]
relevant_subgroups = []
df_metrics = td.extract_metrics(
    text=["this is an example prompt"], 
    lang="en", 
    metrics=metric_group,
    spacy_model="en_core_web_sm"
).drop(columns=["text"] + relevant_subgroups if relevant_subgroups else ["text"])
df_metrics.columns

ℹ Both a spacy model and a language were provided. Will use the spacy
model and ignore language.


Index(['token_length_mean', 'token_length_median', 'token_length_std',
       'sentence_length_mean', 'sentence_length_median', 'sentence_length_std',
       'syllables_per_token_mean', 'syllables_per_token_median',
       'syllables_per_token_std', 'n_tokens', 'n_unique_tokens',
       'proportion_unique_tokens', 'n_characters', 'n_sentences'],
      dtype='object')

Next, we will be working on converting the `text-descriptives` output to a format that can be used to configure the metadata-properties for our supported types: `TermsMetadataProperty`, `IntegerMetadataProperty` and `FloatMetadataProperty`. Note that we are also applying some subjective formatting choices to ensure that the metadata-properties are easy to read and understand.

In [113]:
def clean_column_name(col_name):
    """Clean a column name to fit a specific regex pattern."""
    col_name = col_name.lower()  # Convert to lowercase
    col_name = re.sub(r'[^a-z0-9_]', '_', col_name)  # Replace non-alphanumeric characters with underscores
    return col_name

def create_metadata_properties(df, prefix):
    """Generate metadata properties based on dataframe columns and data types."""
    properties = []
    for col, dtype in df.dtypes.items():
        name = f"{prefix}_{clean_column_name(col)}"
        title = name.replace('_', ' ').title()

        if dtype == 'object':
            prop = rg.TermsMetadataProperty(name=name, title=title)
        elif dtype == 'int64':
            prop = rg.IntegerMetadataProperty(name=name, title=title)
        elif dtype == 'float64':
            prop = rg.FloatMetadataProperty(name=name, title=title)
        elif dtype == 'bool':
            prop = rg.TermsMetadataProperty(name=name, title=title)
        else:
            print(f"Unhandled data type for column {col}: {dtype}")
            continue
        properties.append(prop)
    return properties

metadata_properties = []
metadata_properties += create_metadata_properties(df_metrics, 'prompt')
metadata_properties += create_metadata_properties(df_metrics, 'response')
for metadata_property in metadata_properties:
    try:
        field = ds_remote.metadata_property_by_name(metadata_property.name)
        if not field:
            ds_remote.add_metadata_property(metadata_property)
    except (KeyError, ValueError) as e:
        ds_remote.add_metadata_property(metadata_property)        
ds_remote.metadata_properties

[RemoteFloatMetadataProperty(id=UUID('9691b808-c85e-4248-98bd-89bc2a81b2eb'), client=<httpx.Client object at 0x11105d3d0>, name='response_token_length_mean', title='Response Token Length Mean', visible_for_annotators=True, type='float', min=None, max=None),
 RemoteFloatMetadataProperty(id=UUID('949d3bf5-a8ec-47c8-a4ce-249a9ee4a922'), client=<httpx.Client object at 0x11105d3d0>, name='response_token_length_median', title='Response Token Length Median', visible_for_annotators=True, type='float', min=None, max=None),
 RemoteFloatMetadataProperty(id=UUID('43c7f5a7-e856-4794-9da8-6f89848a3a01'), client=<httpx.Client object at 0x11105d3d0>, name='response_token_length_std', title='Response Token Length Std', visible_for_annotators=True, type='float', min=None, max=None),
 RemoteFloatMetadataProperty(id=UUID('4dcca678-229c-43e8-b0d0-9f5fee2ec902'), client=<httpx.Client object at 0x11105d3d0>, name='response_sentence_length_mean', title='Response Sentence Length Mean', visible_for_annotators=T

### Data collection

In [120]:
dataset = load_dataset("zetavg/ShareGPT-Processed")
dataset = dataset["train"]
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'conversations', 'lang'],
        num_rows: 90665
    })
})

In [130]:
dataset = dataset.filter(function=lambda x: x.get("lang", "?") == "en")
dataset = dataset.filter(lambda x: x["conversations"][0]["from"] == "human")
dataset = dataset.filter(lambda x: len(x["conversations"])>1)
dataset

Filter: 100%|██████████| 9962/9962 [00:02<00:00, 4883.42 examples/s]


Dataset({
    features: ['id', 'conversations', 'lang'],
    num_rows: 9772
})

In [132]:
dataset = dataset.shuffle(seed=42)
dataset = dataset.select(list(range(min(10000, len(dataset))))
dataset

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 dataset = dataset.shuffle(seed=42)                                                           │
│ ❱ 2 dataset = dataset.select(min(list(range(10000)), len(dataset)))                              │
│   3 dataset                                                                                      │
│   4                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
TypeError: '<' not supported between instances of 'int' and 'list'

In [129]:
dataset = dataset.map(lambda x: {"prompt": x["conversations"][0]["value"], "response": x["conversations"][1]["value"]})
dataset

Map:   0%|          | 14/9962 [00:00<00:11, 846.82 examples/s]


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 dataset = dataset.filter(lambda x: x["conversations"][0]["from"] == "human")                 │
│ ❱ 2 dataset = dataset.map(lambda x: {"prompt": x["conversations"][0]["value"], "response": x     │
│   3 dataset                                                                                      │
│   4                                                                                              │
│                                                                                                  │
│ /Users/davidberenstein/Documents/programming/argilla/awesome-argilla-datasets/.env/lib/python3.9 │
│ /site-packages/datasets/arrow_dataset.py:592 in wrapper                                          │
│                                                                                                  │
│    589 │   │   else:                                                                             │
│    590 │   │   │   self: "Dataset" = kwargs.pop("self")                                          │
│    591 │   │   # apply actual function                                                           │
│ ❱  592 │   │   out: Union["Dataset", "DatasetDict"] = func(self, *args, **kwargs)                │
│    593 │   │   datasets: List["Dataset"] = list(out.values()) if isinstance(out, dict) else [ou  │
│    594 │   │   for dataset in datasets:                                                          │
│    595 │   │   │   # Remove task templates if a column mapping of the template is no longer val  │
│                                                                                                  │
│ /Users/davidberenstein/Documents/programming/argilla/awesome-argilla-datasets/.env/lib/python3.9 │
│ /site-packages/datasets/arrow_dataset.py:557 in wrapper                                          │
│                                                                                                  │
│    554 │   │   │   "output_all_columns": self._output_all_columns,                               │
│    555 │   │   }                                                                                 │
│    556 │   │   # apply actual function                                                           │
│ ❱  557 │   │   out: Union["Dataset", "DatasetDict"] = func(self, *args, **kwargs)                │
│    558 │   │   datasets: List["Dataset"] = list(out.values()) if isinstance(out, dict) else [ou  │
│    559 │   │   # re-apply format to the output                                                   │
│    560 │   │   for dataset in datasets:                                                          │
│                                                                                                  │
│ /Users/davidberenstein/Documents/programming/argilla/awesome-argilla-datasets/.env/lib/python3.9 │
│ /site-packages/datasets/arrow_dataset.py:3097 in map                                             │
│                                                                                                  │
│   3094 │   │   │   │   │   total=pbar_total,                                                     │
│   3095 │   │   │   │   │   desc=desc or "Map",                                                   │
│   3096 │   │   │   │   ) as pbar:                                                                │
│ ❱ 3097 │   │   │   │   │   for rank, done, content in Dataset._map_single(**dataset_kwargs):     │
│   3098 │   │   │   │   │   │   if done:                                                          │
│   3099 │   │   │   │   │   │   │   shards_done += 1                                              │
│   3100 │   │   │   │   │   │   │   logger.debug(f"Finished 

In [140]:
# Extract metrics
spacy_model = "en_core_web_md" # we need a model with vectors
df_prompt = td.extract_metrics(text=dataset["prompt"], metrics=metric_group, spacy_model=spacy_model).drop(columns=['text'])
df_response = td.extract_metrics(text=dataset["response"], metrics=metric_group, spacy_model=spacy_model).drop(columns=['text'])

# Identify integer and boolean columns for prompts and responses
int_cols_prompts = df_prompt.select_dtypes(include=['int64']).columns.tolist()
bool_cols_prompts = df_prompt.select_dtypes(include=['boolean']).columns.tolist()

int_cols_responses = df_response.select_dtypes(include=['int64']).columns.tolist()
bool_cols_responses = df_response.select_dtypes(include=['boolean']).columns.tolist()

# Combine column lists for prompts and responses
int_cols = list(set(int_cols_prompts + int_cols_responses))
bool_cols = list(set(bool_cols_prompts + bool_cols_responses))
int_cols, bool_cols

(['n_sentences', 'n_characters', 'n_unique_tokens', 'n_tokens'], [])

Next, we will be casting the `numpy`-datatypes to basic Python built-in datatypes. This is required because the Argilla client doesn't support `numpy`-datatypes.

In [142]:
# --- Functions ---
def cast_to_python_types(df):
    """
    Convert integer and boolean columns to Python native types.
    """
    for column in df.columns:
        df[column].fillna(0, inplace=True)
        if df[column].dtype == bool:
            df[column] = df[column].astype(str)
        elif df[column].dtype == np.int64:
            df[column] = df[column].astype(int)
        elif df[column].dtype == np.float64:
            df[column] = df[column].astype(float)
        else:
            print(f"Unhandled data type for column {column}: {df[column].dtype}")
    return df

df_prompt = cast_to_python_types(df_prompt)
df_response = cast_to_python_types(df_response)

Lastly, we will loop through the Hugging Face dataset, add the metadata-properties and update the Argilla dataset with the new records.

In [184]:
# Prepare feedback records with metadata and suggestions
records = []

cols_with_values_other_than_zeros_or_nan_prompt = df_prompt.columns[~(df_prompt.fillna(0) == 0).all() & ~df_prompt.isnull().any()].tolist()
cols_with_values_other_than_zeros_or_nan_response = df_response.columns[~(df_response.fillna(0) == 0).all() & ~df_response.isnull().any()].tolist()


for i, record in enumerate(dataset):
    # Prepare metadata for prompts
    metadata_prompts = {f"prompt_{col}": value for col, value in df_prompt[cols_with_values_other_than_zeros_or_nan_prompt].iloc[i].items()}
    # Prepare metadata for responses
    metadata_response = {f"response_{col}": value for col, value in df_response[cols_with_values_other_than_zeros_or_nan_response].iloc[i].items()}

    # Explicitly cast integers using Python's native int type
    for col in int_cols:
        if f"prompt_{col}" in metadata_prompts:
            metadata_prompts[f"prompt_{col}"] = int(metadata_prompts[f"prompt_{col}"])
        if f"response_{col}" in metadata_response:
            metadata_response[f"response_{col}"] = int(metadata_response[f"response_{col}"])

    # Convert booleans to strings using Python's native str type
    for col in bool_cols:
        if f"prompt_{col}" in metadata_prompts:
            metadata_prompts[f"prompt_{col}"] = str(metadata_prompts[f"prompt_{col}"])
        if f"response_{col}" in metadata_response:
            metadata_response[f"response_{col}"] = str(metadata_response[f"response_{col}"])

    # Combine both metadata dictionaries into one
    metadata = {**metadata_prompts, **metadata_response}
    record = rg.FeedbackRecord(
        fields={"prompt": record["prompt"]},
        metadata=metadata,
        suggestions=[{"question_name": "response", "value": record["response"]}]
    )
    records.append(record)

# Add records to the dataset and push to Argilla
ds_remote.add_records(records)

Pushing records to Argilla...: 100%|██████████| 1/1 [00:00<00:00, 13.42it/s]


## Add suggestions

For adding the suggestions we will show code for using `langchain` with `ollama` or `llama.cpp`. For my local tests on Apple silicon M1, `ollama` inference was much quicker. Other alternatives might be `llm-studio` or `vllm`.

### Using `ollama` 

We will use [Ollama](https://ollama.ai/) and setup and download the required models.

In [ ]:
!brew install ollama
!ollama serve # start general serving endpoint
!ollama run llama2 # download an run llam2
!ollama run mistral # download an run mistral

Next, we will the `ollama`-client to run local inference on LLMs. This will allow us to run inference on the LLMs without having to send the data to a remote server. This is important because we don't want to send the data to a remote server for privacy reasons.

In [21]:
from langchain.llms import Ollama

temperature = 0.0
llama2 = Ollama(model="llama2", temperature=temperature)
mistral = Ollama(model="mistral", temperature=temperature)

## `llama.cpp` for LLM suggestions

We first need to download the required form for Mistral and Llama2. To do this, we'll use the GGUF (GPT-Generated Unified Format) and one of the quantized models that can be found from [TheBloke](https://huggingface.co/TheBloke). For Mistral, we'll use the [mistral-7b-v0.1.Q4_K_M.gguf](https://huggingface.co/TheBloke/Mistral-7B-v0.1-GGUF/resolve/main/mistral-7b-v0.1.Q4_K_M.gguf?download=true) and for LLama2 we'll use the [llama-2-7b.Q4_K_M.gguf](https://huggingface.co/TheBloke/Llama-2-7B-GGUF/resolve/main/llama-2-7b.Q4_K_M.gguf?download=true).

In [3]:
import os

current_dir = os.getcwd()
local_folder = os.path.join(current_dir, "models")
if not os.path.exists(local_folder):
    os.makedirs(local_folder)

if not os.path.exists(os.path.join(local_folder, "llama-2-7b.Q4_K_M.gguf")):
    !wget -P {local_folder} https://huggingface.co/TheBloke/Llama-2-7B-GGUF/resolve/main/llama-2-7b.Q4_K_M.gguf

if not os.path.exists(os.path.join(local_folder, "mistral-7b-v0.1.Q4_K_M.gguf")):
    !wget -P {local_folder} https://huggingface.co/TheBloke/Mistral-7B-v0.1-GGUF/resolve/main/mistral-7b-v0.1.Q4_K_M.gguf

In [4]:
from langchain.llms import LlamaCpp

temperature=0
llama2 = LlamaCpp(
    model_path="./models/llama-2-7b.Q4_K_M.gguf",
    temperature=temperature,
)
mistral = LlamaCpp(
    model_path="./models/mistral-7b-v0.1.Q4_K_M.gguf",
    temperature=temperature,
)

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from ./models/llama-2-7b.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: - tensor    0:                token_embd.weight q4_K     [  4096, 32000,     1,     1 ]
llama_model_loader: - tensor    1:           blk.0.attn_norm.weight f32      [  4096,     1,     1,     1 ]
llama_model_loader: - tensor    2:            blk.0.ffn_down.weight q6_K     [ 11008,  4096,     1,     1 ]
llama_model_loader: - tensor    3:            blk.0.ffn_gate.weight q4_K     [  4096, 11008,     1,     1 ]
llama_model_loader: - tensor    4:              blk.0.ffn_up.weight q4_K     [  4096, 11008,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_norm.weight f32      [  4096,     1,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.attn_k.weight q4_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    7:         blk.0.attn_output.weight q4_K     [  4096,  4096,     1,     1 ]
llam

### Rating suggestions 

In [90]:
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from pydantic import BaseModel, Field, validator
from langchain.output_parsers import RetryWithErrorOutputParser

# Define your desired data structure.
class RatingScale(BaseModel):
    rating: int = Field(
        description="Describes the numeric quality of a text as an integer from 1 to 7, where 1 equals 'very bad' and 7 equals 'very good'.",
    )
    
    @validator('rating')
    def rating_must_be_between_1_and_7(cls, v):
        if v < 1 or v > 7:
            raise ValueError('The rating must be between 1 and 7.')
        return v

# Set up a parser + inject instructions into the prompt template.
rating_parser = PydanticOutputParser(pydantic_object=RatingScale)

rating_prompt = PromptTemplate(
    template=(
        "Provide a numeric quality rating of a text based on the grammar and phrasing. " 
        "{format_instructions}\n\ntext:{query}"
    ),
    input_variables=["query"],
    partial_variables={"format_instructions": rating_parser.get_format_instructions()},
)

rating_input = rating_prompt.format_prompt(query="How would I become a wizard in the world of Harry Potter?")

output = mistral(rating_input.to_string())
result = rating_parser.parse(output)
result

RatingScale(rating=6)

In [95]:
def get_rating(query):
    try:
        rating_input = rating_prompt.format_prompt(query=query)
        output = mistral(rating_input.to_string())
        result = rating_parser.parse(output)
        return result.rating
    except Exception as e:
        return get_rating(query)
    
get_rating("How would I become a wizard in the world of Harry Potter?")

4

### Prompt suggestions 

In [100]:
class PromptType(BaseModel):
    generation: bool = Field(description="A text that indicates the need for open generation.", default=False)
    rewrite: bool = Field(description="A text that is rewritten by the model.", default=False)
    extract: bool = Field(description="A text that is answered by extracted from the context in the prompt.", default=False)
    closed_qa: bool = Field(description="A text that is answered by the model based on the context.", default=False)
    open_qa: bool = Field(description="A text that is answered by the model without any context.", default=False)
    classification: bool = Field(description="A text that asks to classify.", default=False)
    summarization: bool = Field(description="A text to summarize information with context.", default=False)
    brainstorming: bool = Field(description="A text to list and discuss ideas or concepts.", default=False)
    chat: bool = Field(description="A text that could be used as chat between two people.", default=False)
    code: bool = Field(description="A text about code or programming.", default=False)
    other: bool = Field(description="A text that does not fit into any of the above categories.", default=False)

# Set up a parser + inject instructions into the prompt template.
prompt_type_parser = PydanticOutputParser(pydantic_object=PromptType)

type_select_prompt = PromptTemplate(
    template=(
        "Classify a text into  one category. " 
        "{format_instructions} \n\text:{query}"
    ),
    input_variables=["query"],
    partial_variables={"format_instructions": prompt_type_parser.get_format_instructions()},
)

type_select_input = type_select_prompt.format_prompt(query="I want to buy a new car, what things should I look out for?")

output = mistral(type_select_input.to_string())
prompt_type_parser.parse(output)

PromptType(generation=True, rewrite=True, extract=True, closed_qa=False, open_qa=False, classification=True, summarization=False, brainstorming=False, chat=False, code=False, other=False)

In [99]:
def get_prompt_type(query):
    try:
        type_select_input = type_select_prompt.format_prompt(query=query)
        output = mistral(type_select_input.to_string())
        result = prompt_type_parser.parse(output)
        return result
    except Exception as e:
        return get_prompt_type(query)

get_prompt_type("How to loop through a list of inteegers?")

PromptType(generation=False, rewrite=False, extract=False, closed_qa=False, open_qa=False, classification=False, summarization=False, brainstorming=False, chat=False, code=False, other=False)

### Toxicity suggestions 

In [77]:
class Toxicity(BaseModel):
    illegal: bool = Field(description="illegal: Illegal activities such as terrorism, child abuse or fraud.", default=False)
    harmfull: bool = Field(description="harmfull: Hateful, harassing or violent content such as discrimination, self-harm or bullying.", default=False)
    advice: bool = Field(description="advice: Unqualified advice for instance in legal, medical or financial domains.", default=False)
    
# Set up a parser + inject instructions into the prompt template.
toxicity_parser = PydanticOutputParser(pydantic_object=Toxicity)

toxicity_prompt = PromptTemplate(
    template=(
        "You're given a list of moderation categories as below:\n"
        "- illegal: Illegal illegal activities such as terrorism, child abuse or fraud."
        "- harmfull: Hateful, harassing or violent content such as discrimination, self-harm or bullying."
        "- advice: Unqualified advice for instance in legal, medical or financial domains."
        "Please classify the following text into one of these categories, and answer with a format. \n\nformat: {format_instructions} \n\text: {query}"
    ),
    input_variables=["query"],
    partial_variables={"format_instructions": toxicity_parser.get_format_instructions()},
)
toxicity_input = toxicity_prompt.format_prompt(query="Could you tell me how to evade taxes?")

output = mistral(toxicity_input.to_string(), temperature=0.01)
toxicity_parser.parse(output)

Toxicity(illegal=False, harmfull=True, advice=False)